In [90]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [91]:
folder_path = "data/2023"
FILE_NAME = "merged_sales_fixed.csv"
OLD_FILE_NAME = "geocoded_enriched_merged_sales.csv"

In [92]:
df_new = pd.read_csv(f"{folder_path}/{FILE_NAME}")
df_old = pd.read_csv(f"{folder_path}/{OLD_FILE_NAME}")

# remove 'unit' column from df_old
if 'unit' in df_old.columns:
    df_old = df_old.drop(columns=['unit'])

In [93]:
columns_in_old_but_not_in_new = set(df_old.columns) - set(df_new.columns)
print(columns_in_old_but_not_in_new)

{'y', 'full_address', 'x', 'owner_property_count', 'total_room_num', 'half_bathrooms', 'fireplaces', 'bathrooms', 'exterior_condition', 'year_built', 'properties', 'interior_condition', 'heat_type', 'kitchens', 'owner_name', 'bedrooms', 'ac_type', 'parking_spots', 'foundation'}


In [94]:
df_old

,sale_date,living_area,category,sale_price,parcel,price_per_sf,street_no,street_name,total_room_num,bedrooms,...,parking_spots,year_built,exterior_condition,foundation,full_address,owner_name,owner_property_count,x,y,properties
0,3/31/2023,"1,350",single_family,"600,000",01‐01789‐000,444.44,141,ASHLEY ST,8.0,2.0,...,3.0,1968,Good,Concrete,141 ASHLEY ST BOSTON MA 02128,141 ASHLEY LLC,1.0,-71.010458,42.389219,NaN
1,5/26/2023,"1,401",single_family,"600,000",01‐00931‐000,428.27,606,Bennington ST,7.0,4.0,...,2.0,1900,Average,Stone,606-608 BENNINGTON ST BOSTON MA 02128,OLIVEIRA GABRIEL A,1.0,-71.017048,42.383188,NaN
2,4/28/2023,"1,654",single_family,"649,900",01‐01041‐000,392.82,682,Bennington ST,7.0,3.0,...,0.0,1900,Average,Brick,682 BENNINGTON ST BOSTON MA 02128,MURALLES INGRID J,1.0,-71.013698,42.384396,NaN
3,5/31/2023,"1,568",single_family,"619,000",01‐01521‐001,394.77,1203,Bennington ST,7.0,4.0,...,0.0,1910,Average,Brick,1203 BENNINGTON ST BOSTON MA 02128,MORALES CARDONA DIEGO,1.0,-70.998107,42.389161,NaN
4,3/31/2023,"1,780",single_family,"400,000",01‐04320‐000,224.72,172,Cowper ST,8.0,4.0,...,0.0,1910,Poor,Brick,172 COWPER ST BOSTON MA 02128,TEMPO L.J. CORP,1.0,-71.013738,42.382003,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3156,11/27/2023,"2,814",two_family,"1,000,000",22‐04490‐000,355.37,12,NONANTUM ST,14.0,6.0,...,2.0,1910,Good,Stone,12 NONANTUM ST BOSTON MA 02135,12 NONANTUM LLC,1.0,-71.168387,42.350199,NaN
3157,9/28/2023,"2,325",two_family,"875,000",22‐03741‐000,376.34,135,NONANTUM ST,10.0,5.0,...,0.0,1890,Good,Stone,135-137 NONANTUM ST BOSTON MA 02135,MCCARTHY PATRICIA,3.0,-71.174358,42.350243,NaN
3158,4/14/2023,"2,208",two_family,"900,000",22‐03853‐000,407.61,48,OAK SQUARE AV,12.0,5.0,...,3.0,1890,Average,Stone,48 OAK SQUARE AV BOSTON MA 02135,48 OAK SQUARE AVENUE LLC,1.0,-71.163746,42.351469,NaN
3159,3/30/2023,"2,268",two_family,"875,000",22‐04322‐000,385.80,96,Parsons ST,8.0,4.0,...,4.0,1935,Average,Concrete Block,96-98 PARSONS ST BOSTON MA 02135,CHOWDHURY ISMAIL,1.0,-71.154403,42.353620,NaN


In [95]:
# we will create a temp column for unique id
df_old['temp_unique_id'] = df_old["sale_date"] + "_" + df_old["parcel"]
df_new['temp_unique_id'] = df_new["sale_date"] + "_" + df_new["parcel"]

# strip spaces
df_old['temp_unique_id'] = df_old['temp_unique_id'].str.strip()
df_new['temp_unique_id'] = df_new['temp_unique_id'].str.strip()

# change the temp_unique_id to string
df_old['temp_unique_id'] = df_old['temp_unique_id'].astype(str)
df_new['temp_unique_id'] = df_new['temp_unique_id'].astype(str)


df_old['temp_unique_id'].head()

import unicodedata as ud
import re

def normalise_id(s: str) -> str:
    # 1) Unicode normalisation (breaks compatibility glyphs into base chars)
    s = ud.normalize('NFKD', s)
    # 2) Replace any “fancy” dash characters with ASCII hyphen‑minus
    s = re.sub(r'[\u2010-\u2015\u2212\uFE58\uFE63\uFF0D]', '-', s)
    # 3) Strip any stray whitespace
    return s.strip()

df_new['temp_unique_id_norm'] = df_new['temp_unique_id'].map(normalise_id)
df_old['temp_unique_id_norm'] = df_old['temp_unique_id'].map(normalise_id)

In [96]:
df_new['temp_unique_id_norm'].head()

0    3/31/2023_01-01789-000
1    5/26/2023_01-00931-000
2    4/28/2023_01-01041-000
3    5/31/2023_01-01521-001
4    3/31/2023_01-04320-000
Name: temp_unique_id_norm, dtype: object

In [97]:
print(df_new['temp_unique_id_norm'][0], df_old['temp_unique_id_norm'][0])
df_new['temp_unique_id_norm'][0] == df_old['temp_unique_id_norm'][0]

3/31/2023_01-01789-000 3/31/2023_01-01789-000


True

In [98]:

if not 'temp' in columns_in_old_but_not_in_new:
    columns_in_old_but_not_in_new = list(columns_in_old_but_not_in_new) + ['temp_unique_id_norm']
# move columns from df_old to df_new but based on temp_unique_id
df_merge = pd.merge(df_new, df_old[columns_in_old_but_not_in_new], on='temp_unique_id_norm', how='left')

df_merge.head()





,sale_date,sale_price,parcel,street_no,living_area,category,price_per_sf,street_name,temp_unique_id,temp_unique_id_norm,...,year_built,properties,interior_condition,heat_type,kitchens,owner_name,bedrooms,ac_type,parking_spots,foundation
0,3/31/2023,600000.0,01-01789-000,141.0,1350.0,single_family,444.44,ASHLEY ST,3/31/2023_01-01789-000,3/31/2023_01-01789-000,...,1968,NaN,Average,Electric,2.0,141 ASHLEY LLC,2.0,NaN,3.0,Concrete
1,5/26/2023,600000.0,01-00931-000,606.0,1401.0,single_family,428.27,Bennington ST,5/26/2023_01-00931-000,5/26/2023_01-00931-000,...,1900,NaN,Average,Ht Water/Steam,1.0,OLIVEIRA GABRIEL A,4.0,NaN,2.0,Stone
2,4/28/2023,649900.0,01-01041-000,682.0,1654.0,single_family,392.82,Bennington ST,4/28/2023_01-01041-000,4/28/2023_01-01041-000,...,1900,NaN,Good,Forced Hot Air,1.0,MURALLES INGRID J,3.0,Central AC,0.0,Brick
3,5/31/2023,619000.0,01-01521-001,1203.0,1568.0,single_family,394.77,Bennington ST,5/31/2023_01-01521-001,5/31/2023_01-01521-001,...,1910,NaN,Average,Forced Hot Air,1.0,MORALES CARDONA DIEGO,4.0,Central AC,0.0,Brick
4,3/31/2023,400000.0,01-04320-000,172.0,1780.0,single_family,224.72,Cowper ST,3/31/2023_01-04320-000,3/31/2023_01-04320-000,...,1910,NaN,Fair,Ht Water/Steam,2.0,TEMPO L.J. CORP,4.0,NaN,0.0,Brick


In [99]:
print(df_merge[columns_in_old_but_not_in_new])

              y                           full_address          x  \
0     42.389219         141  ASHLEY ST BOSTON MA 02128 -71.010458   
1     42.383188  606-608 BENNINGTON ST BOSTON MA 02128 -71.017048   
2     42.384396     682  BENNINGTON ST BOSTON MA 02128 -71.013698   
3     42.389161    1203  BENNINGTON ST BOSTON MA 02128 -70.998107   
4     42.382003         172  COWPER ST BOSTON MA 02128 -71.013738   
...         ...                                    ...        ...   
4952  42.350199        12  NONANTUM ST BOSTON MA 02135 -71.168387   
4953  42.350243    135-137 NONANTUM ST BOSTON MA 02135 -71.174358   
4954  42.351469      48  OAK SQUARE AV BOSTON MA 02135 -71.163746   
4955  42.353620       96-98 PARSONS ST BOSTON MA 02135 -71.154403   
4956  42.354111  40-42 PRESENTATION RD BOSTON MA 02135 -71.171281   

      owner_property_count  total_room_num  half_bathrooms  fireplaces  \
0                      1.0             8.0             2.0         2.0   
1                      

In [100]:
# save as csv
df_merge.to_csv(f"{folder_path}/geocoded_merged_sales_fixed.csv", index=False)